# Semi-invertible autoencoder

In [1]:
import numpy as np
import logging
import sys
from matplotlib import pyplot as plt
import torch

%matplotlib inline
sys.path.append("../")


from aef.models.autoencoding_flow import TwoStepAutoencodingFlow
from aef.trainer import AutoencodingFlowTrainer, NumpyDataset
from aef.losses import nll, mse

logging.basicConfig(
    format="%(asctime)-5.5s %(name)-20.20s %(levelname)-7.7s %(message)s",
    datefmt="%H:%M",
    level=logging.INFO,
)


## Data

In [2]:
x = np.load("../data/tth/x_test.npy")
tth_data = NumpyDataset(x, x)
print(x.shape)
print(tth_data.n)

(10000, 48)
10000


In [3]:
np.all(np.isfinite(x))

True

## Train autoencoder

In [4]:
ae = TwoStepAutoencodingFlow(data_dim=48, latent_dim=8, n_mades_inner=3, n_mades_outer=1)

In [5]:
trainer = AutoencodingFlowTrainer(ae, output_filename="../figures/training/aef_phase1")
trainer.train(
    dataset=tth_data,
    loss_functions=[mse],
    loss_labels=["MSE"],
    loss_weights=[1.],
    batch_size=128,
    epochs=20,
    verbose="all",
    initial_lr=1.e-5,
    final_lr=1.e-7,
)

16:18 aef.trainer          INFO    Epoch   1: train loss      nan (MSE:    nan)
16:18 aef.trainer          INFO               val. loss       nan (MSE:    nan)
16:18 aef.trainer          INFO    Epoch   2: train loss      nan (MSE:    nan)
16:18 aef.trainer          INFO               val. loss       nan (MSE:    nan)
16:18 aef.trainer          INFO    Epoch   3: train loss      nan (MSE:    nan)
16:18 aef.trainer          INFO               val. loss       nan (MSE:    nan)
16:18 aef.trainer          INFO    Epoch   4: train loss      nan (MSE:    nan)
16:18 aef.trainer          INFO               val. loss       nan (MSE:    nan)
16:18 aef.trainer          INFO    Epoch   5: train loss      nan (MSE:    nan)
16:18 aef.trainer          INFO               val. loss       nan (MSE:    nan)
16:18 aef.trainer          INFO    Epoch   6: train loss      nan (MSE:    nan)
16:18 aef.trainer          INFO               val. loss       nan (MSE:    nan)
16:18 aef.trainer          INFO    Epoch

KeyboardInterrupt: 

## Visualize latent space

In [ ]:
n = 1000

x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(n)], dim=0)
y = np.asarray([mnist[i][1] for i in range(n)])

x = x.view(x.size(0), -1)
x.to(self.device, self.dtype)
x_out, _, u = self.model(x)

x = x.detach().numpy().reshape(-1, 28, 28)
x_out = x_out.detach().numpy().reshape(-1, 28, 28)
u = u.detach().numpy().reshape(x_out.shape[0], -1)
tsne = TSNE(n_components=2, verbose=0, perplexity=40, n_iter=300).fit_transform(u)

In [ ]:
fig = plt.figure(figsize=(10,5))

ax = plt.subplot(1,2,1)
for i in range(10):
    plt.scatter(u[y==i][:,0], u[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()
plt.xlabel(r"$u_0$")
plt.ylabel(r"$u_1$")
    
ax = plt.subplot(1,2,2)
for i in range(10):
    plt.scatter(tsne[y==i][:,0], tsne[y==i][:,1], c="C{}".format(i), s=20., label="{}".format(i+1))
plt.legend()
plt.xlabel(r"$t-SNE component 0$")
plt.ylabel(r"$t-SNE component 1$")

plt.tight_layout()
plt.savefig("../figures/mnist_latent.pdf")


## Visualize reconstruction

In [ ]:
x = torch.cat([mnist[i][0].unsqueeze(0) for i in range(1000)], dim=0)
y = np.asarray([mnist[i][1] for i in range(1000)])

In [ ]:
h = ae.encoder(x)
x_out = ae.decoder(h)
h = h.detach().numpy()
x_out = x_out.detach().numpy()

In [ ]:
fig = plt.figure(figsize=(12,12))

for i in range(18):
    ax = plt.subplot(6, 6, 2*i + 1)
    plt.imshow(x[i].reshape((28,28)))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    ax = plt.subplot(6, 6, 2*i + 2)
    plt.imshow(x_out[i].reshape((28,28)))
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
        
plt.tight_layout()
plt.savefig("../figures/reconstruction.pdf")
